# TFM: MAPA DE PELIGROSIDAD

### Nuestro modelo es supervisado ya que tiene una variable objetivo y predecir un valor futuro. 
Modelo de clasificación: Multiclase, tenemos varias.
# PREPROCESADO

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv(r"C:\Users\Brigitte\Documents\master\tfm_dataset\datasets_unificados.csv")

In [10]:
pd.set_option('display.max_columns', None)
#Tipos de variables
print(dataset.info())
print(dataset.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3912398 entries, 0 to 3912397
Data columns (total 55 columns):
 #   Column                            Dtype         
---  ------                            -----         
 0   Zip Code                          float64       
 1   Total                             float64       
 2   Sin Educacion Basica Obligatoria  float64       
 3   Educacion Basica Obligatoria      float64       
 4   Educacion Media Superior          float64       
 5   Educacion Superior                float64       
 6   Master o Doctorado                float64       
 7   Estimated Median Income           float64       
 8   Salario_bajo                      float64       
 9   Salario_medio                     float64       
 10  Salario_alto                      float64       
 11  Nivel Pobreza                     float64       
 12  AREA NAME                         object        
 13  Tasa Desempleo 2021               float64       
 14  Temp Min          

# Una vez hemos analizado las variables vemos que:
 **DR_NO** son códigos los cuales no nos aportan nueva información, ***se podría eliminar. 
 **Date Rptd** time when the crime reported df[time_field], esto no nos interesa así que *****podemos eliminarla
 **DATE OCC** time when the crime occurred time_field +++Tenmos que modificar el tipo de variable a data.
 **Rpt Dist No No sabemos muy bien que es, tenemos que seguir investigando. **
 Part 1-2 Podria tratarse de mañana o tarde, lo comprobaremos. Si no es así la eliminaremos.**** 
 **Crm Cd** En esta variable están los códigos de los delitos. **Crm Cd Desc**  esta su descrippción.
 **Mocodes**, es un código que nos da datos sobre el sospechoso. No, nos interesaría para nuestro caso de estudio. 
 **Vict Age** vamos hacer un rango de edad con esta variable y así personalizar más la aplicación al usuario. 
 **Vict Sex** Hay cinco variables, tenemos que estudiarlas para ver de que elementos se compone. 
** Vict Descent** Tendriamos que ordenarlo de una mejor forma o analizarlo.
 **Premis Cd, Premis Desc** nos dice dónde habia ocurrido el delito, consultar si nos interesa dejarlo o no. ****Eliminarlo
 **Weapon Used Cd y Weapon Desc** Es la arma usada. consultar si nos interesa dejarlo o no. *******
 **Status y Status Desc** Estado del caso. Dejamos solo la descripción. *******Eliminarlo
 **Crm Cd 1,2,3,4 investigar**
 **LOCATION y Cross Street.** Dirección aprox. del delito. ******Eliminarlo
 **LAT y LON.** Long y altitud. Si es muy importante. 

In [6]:
#Valores únicos
print(dataset.nunique())

Zip Code                                92
Total                                   63
Sin Educacion Basica Obligatoria        61
Educacion Basica Obligatoria            62
Educacion Media Superior                62
Educacion Superior                      62
Master o Doctorado                      62
Estimated Median Income                 62
Salario_bajo                             2
Salario_medio                            2
Salario_alto                             2
Nivel Pobreza                           30
AREA NAME                               13
Tasa Desempleo 2021                     59
Temp Min                                 6
Temp Max                                12
Lluvia Ene                               2
Lluvia Feb                               2
Lluvia Mar                               2
Lluvia Abr                               2
Lluvia May                               2
Lluvia Jun                               3
Lluvia Jul                               1
Lluvia Ago 

#### Modificamos las variables que tendrian que estar en fecha a fecha y hora. 

In [7]:
#Fecha como datetime
dataset['DATE OCC'] = pd.to_datetime(dataset['DATE OCC'])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3912398 entries, 0 to 3912397
Data columns (total 55 columns):
 #   Column                            Dtype         
---  ------                            -----         
 0   Zip Code                          float64       
 1   Total                             float64       
 2   Sin Educacion Basica Obligatoria  float64       
 3   Educacion Basica Obligatoria      float64       
 4   Educacion Media Superior          float64       
 5   Educacion Superior                float64       
 6   Master o Doctorado                float64       
 7   Estimated Median Income           float64       
 8   Salario_bajo                      float64       
 9   Salario_medio                     float64       
 10  Salario_alto                      float64       
 11  Nivel Pobreza                     float64       
 12  AREA NAME                         object        
 13  Tasa Desempleo 2021               float64       
 14  Temp Min          

In [8]:
#Hora como bject no se porque...?
from datetime import datetime, timedelta
dataset['TIME OCC'] = dataset['TIME OCC'].apply(lambda x: (datetime.min ).time())
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3912398 entries, 0 to 3912397
Data columns (total 55 columns):
 #   Column                            Dtype         
---  ------                            -----         
 0   Zip Code                          float64       
 1   Total                             float64       
 2   Sin Educacion Basica Obligatoria  float64       
 3   Educacion Basica Obligatoria      float64       
 4   Educacion Media Superior          float64       
 5   Educacion Superior                float64       
 6   Master o Doctorado                float64       
 7   Estimated Median Income           float64       
 8   Salario_bajo                      float64       
 9   Salario_medio                     float64       
 10  Salario_alto                      float64       
 11  Nivel Pobreza                     float64       
 12  AREA NAME                         object        
 13  Tasa Desempleo 2021               float64       
 14  Temp Min          

#### Para tener un mejor conocimiento del dataset podemos analizar los descriptivos 

In [ ]:
dataset.describe()

De esta aproximación podemos concluir las siguientes observaciones:
- edad de 120 max

## Examinamos las posibles variables categóricas (con menos de 10 valores únicos)

Estas variables son:
- Vict Sex
- Part 1-2
- Status
- Status Desc
- Crdm cd 4

Empezamos estudaindo la variable Vict Sex

In [ ]:
for valor in dataset['Vict Sex'].unique():
    print(valor)

En una primera aproximación de los datos podemos observar que tenemos 3 variables a examinar:
- La variable Crdm cd, la cual indica del 1 al 4 el nivel de peligrosidad de los datos, la cual podriamos fusionar, de alguna manera.
- La variable vict sex tiene 5 valores distintos por lo que deberemos examinar cuales son y por que, ya que solo debería tener dos, hombre y mujer.

In [ ]:
contador = dataset['Vict Sex'].value_counts()['H']
contador

In [ ]:
dataset.shape

In [ ]:
contador = dataset['Vict Sex'].value_counts()
contador

In [ ]:
contador = dataset['Vict Sex'].isnull().sum()
contador

In [ ]:
# Número de valores distintos por variable
dataset1 = dataset[dataset['Vict Sex'] != 'H']
dataset1.info()

In [ ]:
for valor in dataset1['Vict Sex'].unique():
    print(valor)

In [ ]:
dataset2 = dataset1[dataset1['Vict Sex'] != '-']
dataset2.info()

In [ ]:
for valor in dataset2['Vict Sex'].unique():
    print(valor)

In [ ]:
contador = dataset2['Vict Sex'].isnull().sum()
contador

In [ ]:
contador = dataset2['Vict Sex'].value_counts()
contador

In [ ]:
import plotly.express as px
# Histograma de calificación del productor
fig = px.histogram(dataset2, x="Vict Sex") 
fig.show()

Hemos eliminado los valores H y - ya que su represenatción era mínima, sin embargo, en el caso de X deberemos determinar si está variable será determinante. Pensamos que puede ser interesante dejar este valor porque actualmente el valor 'otros' es visulizado.

In [ ]:
sns.violinplot(x='Vict Sex', y='Vict Age', data=dataset2, palette='viridis')

2. Estudiamos la variable Status y Status Desc

hacer rangos de edad

In [ ]:
for valor in dataset2['Status'].unique():
    print(valor)

In [ ]:
for valor in dataset2['Status Desc'].unique():
    print(valor)

In [ ]:
contador = dataset2['Status'].value_counts()
contador

In [ ]:
contador = dataset2['Status Desc'].value_counts()
contador

Podemos observar que:
- Las dos variables tienen exactamente los mismos valores y el mismo número, por lo tanto, se trata de la misma variable
- Los valores Unknown representan un pocentaje insignificante y podrán ser eliminados

In [ ]:
dataset2 = dataset1[dataset1['Status Desc'] != 'UNK']
dataset2.info()

In [ ]:
dataset3 = dataset2.drop(['Status'],axis=1)

In [ ]:
import plotly.express as px
# Histograma de calificación del productor
fig = px.histogram(dataset3, x="Status Desc") 
fig.show()

3. Part 1-2:

In [ ]:
for valor in dataset2['Part 1-2'].unique():
    print(valor)

In [ ]:
# Calcula la matriz de correlación
correlation_matrix = dataset3.corr()

# Crea un mapa de correlación utilizando Seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Mapa de Correlación')
plt.show()

In [ ]:
for valor in dataset2['Crm Cd'].unique():
    print(valor)

¿eliminamos crm cd y part 1-2?

## Pasar a factor

In [ ]:
# Lista de columnas con menos de 10 valores distintos. Potenciales factores!
dataset3['Vict Sex'] = dataset3['Vict Sex'].astype('category')
dataset3['Status Desc'] = dataset3['Status Desc'].astype('category')

In [ ]:
dataset3.info()
dataset3.describe()

In [ ]:
dataset3.describe(exclude=np.number)

In [ ]:
for valor in dataset2['Premis Desc'].unique():
    print(valor)

### Creamos los rangos de edad:

In [ ]:
## AGE (realizamos una variable con 3 rangos de edad y 3 variables con esos mismos rangos)
dataset3['Niño'] = dataset3['Vict Age'].apply(lambda x: 1 if x<=12 else 0)
dataset3['Joven'] = dataset3['Vict Age'].apply(lambda x: 1 if x>12 and x<=26 else 0)
dataset3['Adulto'] = dataset3['Vict Age'].apply(lambda x: 1 if x>26 and x<60 else 0)
dataset3['Mayor de 60'] = dataset3['Vict Age'].apply(lambda x: 1 if x>=60 else 0)
# dataset3['Mayor de 60'] = dataset3['Vict Age'].apply(lambda x: 0 if x<=35 else (1 if x>35 and x<60 else(2)))


In [ ]:
pd.set_option('display.max_columns', None)

dataset3.head()

### Valores atípicos

In [ ]:
dataset3.select_dtypes(include=np.number).apply(lambda x: x.skew())

In [ ]:
dataset3[['Crm Cd 4']].nunique()

In [ ]:
dataset['Crm Cd 1'].isnull().sum()

In [ ]:
dataset3['Crm Cd'].isnull().sum()

In [ ]:
print(dataset3['Crm Cd 1'].value_counts(),
dataset3['Crm Cd 2'].value_counts(),
dataset3['Crm Cd 3'].value_counts(),
dataset3['Crm Cd 4'].value_counts())

In [ ]:
dataset_prueba= dataset3[['Crm Cd','Crm Cd Desc']]
dataset_prueba

In [ ]:
dataset_ordenado = dataset_prueba.sort_values(by='Crm Cd', ascending=True)

In [ ]:
dataset_ordenado

In [ ]:
dataset_sin_duplicados = dataset_ordenado.drop_duplicates(subset=['Crm Cd'])


In [ ]:
dataset_sin_duplicados.shape

In [ ]:
pd.set_option('display.max_rows', None)
dataset_sin_duplicados

In [ ]:
dataset3['Crm Cd 1'].info()

In [ ]:
dataset3.describe()

In [ ]:
# Convertir las columnas en conjuntos y encontrar los valores distintos
valores_distintos_col1 = set(dataset['Crm Cd'])
valores_distintos_col2 = set(dataset['Crm Cd 1'])

# Encontrar los valores distintos entre las dos columnas
valores_distintos = valores_distintos_col1.symmetric_difference(valores_distintos_col2)

print("Valores distintos entre las columnas:", valores_distintos)

In [ ]:
dataset['Crm Cd 1'].value_counts()[521]

In [ ]:
dataset['Crm Cd 1'].value_counts()[430]

In [ ]:
prueba2 = dataset3.drop_duplicates(subset=['Crm Cd'])


In [ ]:
prueba1= dataset3.drop_duplicates(subset=['Crm Cd 2'])

In [ ]:
prueba1.shape

In [ ]:
# Verificar los valores únicos en 'Crm Cd 2'
crm_cd_2_unique = dataset3['Crm Cd 2'].unique()

# Verificar los valores únicos en 'Crm Cd'
crm_cd_unique = dataset3['Crm Cd'].unique()

# Encontrar los valores en 'Crm Cd 2' que no están en 'Crm Cd'
valores_no_en_crm_cd = [valor for valor in crm_cd_2_unique if valor not in crm_cd_unique]

# Imprimir los valores que no están en 'Crm Cd'
print("Valores en 'Crm Cd 2' que no están en 'Crm Cd':")
print(valores_no_en_crm_cd)

In [ ]:
prueba2.shape


In [ ]:
dataset3['Crm Cd 2'].value_counts()[521.0]

In [ ]:
dataset_sin_duplicados

In [ ]:
# Convertir las columnas en conjuntos y encontrar los valores distintos
valores_distintos_col3 = set(prueba1)
valores_distintos_col4 = set(prueba2)


# Encontrar los valores distintos entre las dos columnas
valores_distintos = valores_distintos_col3.symmetric_difference(valores_distintos_col4)

print("Valores distintos entre las columnas:", valores_distintos)

In [ ]:
dataset3.describe()


In [ ]:
dataset3.count()
#dataset3['Crm Cd 2'].value_counts()

In [ ]:
# FUNCIÓN PARA ASIGNAR EL NIVEL DE GRAVEDAD DE LOS DELITOS. 1 infracción 5 muy muy grave.

def asignar_gravedad(valor):
    if valor in [110, 121, 235, 900, 907, 921, 860, 822, 910, 920]:
        return 5
    elif valor in [113, 122, 210, 220, 230, 231,236, 237, 250, 251, 354, 433, 434, 940, 942, 944,
                                           954, 956, 470, 473, 870, 845, 840, 820, 815, 814, 626, 627, 648, 805, 762, 761,
                                         760, 756, 755]:
        return 4
    elif valor in [310,330, 331, 341, 343, 345, 347, 349, 410, 420, 421, 901, 902, 435, 
                                                 903, 436, 904, 931, 943, 446, 350, 450, 946, 948, 949, 950, 951, 475, 622,
                                                865, 830, 821, 625, 624, 647, 810, 649, 763, 661, 662, 668, 740]:
        return 3
    elif valor in [320, 351, 352, 353, 432, 437, 438, 906, 924, 928, 930, 439, 932, 440, 441, 442, 
                                          443, 444, 445, 451, 452, 471,474, 480, 485, 890, 888, 487, 510, 520, 522, 882, 623,
                                          880, 850, 651, 653, 652, 654, 660, 664, 666, 670, 753, 745, 922]:
        return 2
    elif valor in [933, 886, 884, 813, 812, 806]:
        return 1
    else:
        return 0
    
dataset3['nivel_gravedad'] = dataset3['Crm Cd'].apply(asignar_gravedad)

In [ ]:
dataset3["nivel_gravedad"].value_counts()

In [ ]:
nivel_gravedad_0 = dataset3[dataset3['nivel_gravedad']==0]
valores_Crm_Cd = nivel_gravedad_0['Crm Cd']
valores_Crm_Cd.unique()
#print(valores_Crm_Cd)

In [ ]:
dataset3.head(10)

In [ ]:
dataset3["nivel_gravedad"].info()

In [ ]:
columnas_a_eliminar = ['Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4' ]

# Eliminar las columnas especificadas
dataset3 = dataset3.drop(columns=columnas_a_eliminar)
dataset3.head(10)

In [ ]:
# Calcular el coeficiente de correlación
correlacion = dataset3['Part 1-2'].corr(dataset3['nivel_gravedad'])

print(f"Coeficiente de correlación: {correlacion}")

In [ ]:
# La correlación que nos da entre parte 1 o 2 y los diferentes niveles de gravedad es muy baja o incluso negativa. 
correlacion = (dataset3['Part 1-2']==1).corr(dataset3['nivel_gravedad']==5)

print(f"Coeficiente de correlación: {correlacion}")

In [ ]:
#Eliminamos 'Part 1-2' ya que su correlación es muy baja y no aporta mucha información. comprobar!!!!!! head tampoco nos deja 
dataset4 = dataset3.drop('Part 1-2', axis=1, inplace=True)
print(dataset4)

In [ ]:
# Calcula la matriz de correlación
correlation_matrix = dataset3.corr()

# Crea un mapa de correlación utilizando Seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Mapa de Correlación')
plt.show()

In [ ]:
#Comprobar normalidad
import plotly.express as px
# Histograma de calificación del productor
fig = px.histogram(dataset3, x= "nivel_gravedad") 
fig.show()
#Por ahora la dejamos en continua. 

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(dataset3, title='Profiling Report')
profile